In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from sklearn.metrics import confusion_matrix
from skimage.transform import rescale, resize, downscale_local_mean
import binvox_rw
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import h5py
import pandas as pd

from extraction_performances import extraction_performances
from IAHOS import IAHOS
from hyperparams_initialization import hyperparams_initialization
import keras
from keras import backend as K
from keras.models import Sequential,Input,Model
from keras.layers import Conv3D,MaxPooling3D,Dense,Flatten,Dropout
from keras import initializers,optimizers,backend as k
from time import time

In [ ]:
data = pd.read_csv("all.csv") 
print (data)

In [ ]:
classes = ['airplane','bin','bag','basket','bathtub','bed','bench','bird house','library',
          'bottle','bowl','bus','dresser','camera','can','cap','car','smartphone','chair',
          'clock','keyboard','dishwashers','display','headphones','faucet','library',
          'guitar','helmet','jar','knife','lamp','laptop','loudspeaker','mail box','unknown',
          'microwave','motorbike','cup','piano','pillow','pistol','plant','scanner','telecontrol',
          'rifle','rocket','skate','sofa','oven','table','smartphone','tower','railway engine',
          'ship','washing machine']
np.save('classes',classes)

In [ ]:
categories = os.listdir('G:\ShapeNetCore.v2')
lengths=[]
dataset=[]
models_number=56
insert = False
for i in tqdm(range(len(categories))):
    model = os.listdir('G:\ShapeNetCore.v2/'+categories[i])
    l=0
    for j in range(len(model)):
        #if l<models_number:
            try:
                with open('G:\ShapeNetCore.v2/'+categories[i]+'/'+model[j]+'/models/'+'model_normalized.surface.binvox', 'rb') as f:
                    solid = binvox_rw.read_as_3d_array(f)
                    if insert==True:
                        Volume = np.zeros((128,128,128))
                        for k in range(len(surface.data[0])):
                            x = surface.data[0][k]
                            y = surface.data[1][k]
                            z = surface.data[2][k]
                            Volume[x,y,z]=1
                        Volume_resized = resize(Volume, (64,64,64),anti_aliasing=True)
                        dataset.append(Volume_resized)
                    l+=1
            except:
                pass
    lengths.append(l)
 

In [ ]:
hf = h5py.File('data64.h5', 'w')
hf.create_dataset('dataset_1', data=dataset)
hf.close()

Plot a model (at random from the dataset)

In [ ]:
x=surface.data[0]
y=surface.data[1]
z=surface.data[2]
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,mode='markers',
                      marker=dict(size=1,color=z,colorscale='hot',opacity=0.5))])
fig.update_layout(width=800,height=400,autosize=True)
fig.show()

Neural Network

In [ ]:
training_samples = 55*56
labels = np.zeros((training_samples,55))
for i in range(training_samples):
    j=int(i/56)
    labels[i,j]=1

In [ ]:
#Load dataset from disk
hf = h5py.File('data32.h5', 'r')
n1 = hf.get('dataset_1')
n1 = np.array(n1)
hf.close()
randomize = np.arange(len(n1))
np.random.shuffle(randomize)
n1=n1[randomize]
labels=labels[randomize]
index1 = int(len(n1)*0.8)
training_set = n1[0:index1].reshape((-1,32,32,32,1))
training_labels=labels[0:index1]
validation_set = n1[index1:].reshape((-1,32,32,32,1))
validation_labels=labels[index1:]

In [ ]:
attempts=2
variables=9
iterations=attempts**variables

In [ ]:
def Classifier(input_size,i,hm,optimizer):
    x = Input(shape=(input_size,input_size,input_size,1))
    x1 = (Conv3D(int(hm[0,i]),kernel_size=(int(hm[1,i]),int(hm[1,i]),int(hm[1,i])),activation='relu'))(x)
    x1 = (MaxPooling3D(pool_size=(int(hm[2,i]),int(hm[2,i]),int(hm[2,i])),
                       strides=(int(hm[3,i]),int(hm[3,i]),int(hm[3,i]))))(x1)
    for j in range(int(hm[4,i])):
        x1 = (Conv3D(int(hm[5,i]),kernel_size=(int(hm[6,i]),int(hm[6,i]),int(hm[6,i])),activation='relu'))(x1)
        #x1 = (MaxPooling3D(pool_size=(2,2,2)))(x1)
    y1 = Flatten()(x1)
    y2 = (Dense(int(hm[7,i]),activation='relu'))(y1)
    y2 = Dropout(hm[8,i])(y2)
    y = (Dense(55,activation='softmax'))(y2)
    classifier=Model(inputs=x,outputs=y)
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

In [ ]:
limits = [[4,16],[2,4],[2,3],[1,2],[0,2],[4,16],[2,4],[55,200],[0,0.5]]
method='grid'
rounds=2
tgp,tgp2,ogp,ogp2=IAHOS(rounds,method,limits,attempts,variables,iterations,Classifier,
                       training_set,training_labels,validation_set,validation_labels)

In [ ]:
fig = make_subplots(rows=2, cols=2,subplot_titles=("Mean train. accuracy first round",
                                                   "Mean valid accuracy first round",
                                                  "Mean train accuracy last round",
                                                  "Mean valid accuracy last round"))
y = ['#filter1','kernel1','maxpool1','stride_pool','#layers',
     '#filter2','kernel2','fully_dim','dropout']
x = np.linspace(0,len(temp_general_perf[0])-1,len(temp_general_perf[0]))
Colorscale = [[0, '#FF0000'],[0.5, '#F1C40F'], [1, '#00FF00']]
fig.add_trace(go.Heatmap(y=[y[i] for i in range(len(y))],
                   x=[0,1],
                   z=old_general_perf2, colorscale = Colorscale),row=1,col=1)
fig.add_trace(go.Heatmap(y=[y[i] for i in range(len(y))],
                   x=[0,1],
                   z=old_general_perf,colorscale=Colorscale),row=1,col=2)
fig.add_trace(go.Heatmap(y=[y[i] for i in range(len(y))],
                   x=x,
                   z=temp_general_perf2, colorscale = Colorscale),row=2,col=1)
fig.add_trace(go.Heatmap(y=[y[i] for i in range(len(y))],
                   x=x,
                   z=temp_general_perf,colorscale=Colorscale),row=2,col=2)
fig.update_layout(height=600, width=800)

fig.show()

In [ ]:
training_accuracy=[]
validation_accuracy=[]
test_scores=[]
optimizers = ['sgd','rmsprop','adagrad','adadelta','adam','adamax','nadam']
classifier=Classifier(32,optimizers[0])
print (classifier.summary())

for optimizer in tqdm(optimizers):
    classifier=Classifier(32,optimizer)
    history=classifier.fit(training_set,training_labels,validation_data=[validation_set,validation_labels],
                           epochs=20,batch_size=10,verbose=0,callbacks=[tensorboard])
    training_accuracy.append(history.history['acc'])
    validation_accuracy.append(history.history['val_acc'])
    j+=1
    if j<7:
        K.clear_session()
    del classifier

In [ ]:
plt.figure(figsize=(12,4))
for i in range(len(optimizers)):
    plt.plot(training_accuracy[i]) 
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(optimizers)
plt.show()

plt.figure(figsize=(12,4))
for i in range(len(optimizers)):
    plt.plot(validation_accuracy[i]) 
plt.title('Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(optimizers)
plt.show()